In [1]:
from rdkit import Chem
from rdkit.Chem import rdMolDescriptors
from rdkit.Chem import Descriptors
from rdkit.Chem.rdMolDescriptors import CalcNumHBA
from rdkit.Chem.rdMolDescriptors import CalcNumHBD

import numpy as np
import pandas as pd

import os

## bring DB --> primary processing

In [2]:
# 석우 gpt 수집 데이터

folder_path = "/home/ufslab223/Shared/KSW/GPT/data_extracted_gpt"

sw_LLM_list = [pd.read_csv(os.path.join(folder_path, f), index_col=False) for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f))]
sw_LLM_list = [db.drop(columns='Unnamed: 0') if 'Unnamed: 0' in db.columns else db for db in sw_LLM_list]
sw_LLM_list = [db.drop(columns='data_ID') if 'data_ID' in db.columns else db for db in sw_LLM_list]
sw_df = pd.concat(sw_LLM_list, axis=0).reset_index(drop=True)



In [3]:
# 아인님 gpt 수집 데이터

folder_path = "/home/ufslab223/Shared/ains/GPT/data_extracted_gpt"

ain_LLM_list = [pd.read_csv(os.path.join(folder_path, f), index_col=False) for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f))]
ain_LLM_list = [db.drop(columns='Unnamed: 0') if 'Unnamed: 0' in db.columns else db for db in ain_LLM_list]
ain_LLM_list = [db.drop(columns = 'data_ID') if 'data_ID' in db.columns else db for db in ain_LLM_list]
ain_df = pd.concat(ain_LLM_list, axis=0).reset_index(drop=True)

/tmp/ipykernel_3877540/992123533.py:5: ParserWarning: Length of header or names does not match length of data. This leads to a loss of data with index_col=False.
  ain_LLM_list = [pd.read_csv(os.path.join(folder_path, f), index_col=False) for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f))]


In [4]:
# 데이터 통일 및 결측치 제거
def is_valid_float(x):
    try:
        float(x)
        return True
    except:
        return False

LLM_df = pd.concat([sw_df, ain_df], axis=0, ignore_index = True)
LLM_df.drop(columns=['compound name', 'publish'], inplace = True)
LLM_df.dropna(subset=['Smiles', 'Rf'], inplace = True)    #Smiles nan 제거,  Rf nan 제거
LLM_df.dropna(subset = ['eluent1', 'eluent2'], how = 'all', inplace = True) #eluent1,2 모두 없는거 제거
mask = mask = LLM_df['Rf'].apply(is_valid_float) # 가끔 Rf 가 문자열로 보관되는데, float으로 변환 가능하지만, 0.4 - 0.45 이런건 안되니 걍 지우자
LLM_df = LLM_df[mask]

# ratio 칼럼의 모두 string 으로 변환
# 밑에서 가공하려고
LLM_df['ratio'] = LLM_df['ratio'].astype(str)

LLM_df.columns = ['Smiles', 'elu1', 'elu2', 'ratio', 'Rf', 'title', 'doi']

print(f'raw개수: {len(LLM_df)}')


raw개수: 23974


In [6]:
'''# 이상한 용액 있는 파일 추적 (용액 이름 이상하게 명명해서 converter에 넣기 부적절한것)
def file_finder_weird_eluent(path, eluent):
    if path == "/home/ufslab223/Shared/KSW/GPT/data_extracted_gpt":
        print('석우 수집: ')
    elif path == '/home/ufslab223/Shared/ains/GPT/data_extracted_gpt':
        print('아인 수집: ')
    a=None
    for file in os.listdir(path):
        if os.path.isfile(os.path.join(path, file)):
            f = pd.read_csv(os.path.join(path, file))
            mask = ((f['eluent1']==eluent) | (f['eluent2']==eluent))
            ff = f[mask]
            if (len(ff)>0):
                a=file
                datafile = pd.read_csv(path+'/'+a)
                print(f'{file} / doi" {datafile.loc[0,'doi']}')
    if a is None:
        print('해당 eluent 있는 파일 없음\n')
        return



# 석우 수집 폴더 추적
path_sw="/home/ufslab223/Shared/KSW/GPT/data_extracted_gpt"
# 아인님 수집 폴더 추적
path_ain='/home/ufslab223/Shared/ains/GPT/data_extracted_gpt'


file_finder_weird_eluent(path_sw,'AT')
file_finder_weird_eluent(path_ain,'AT')'''

석우 수집: 
해당 eluent 있는 파일 없음

아인 수집: 
datatable522.csv / doi" https://pubs.acs.org/doi/10.1021/acs.orglett.2c01674


In [ ]:
'''# convert에 도입하기 부적절한 용액 데이터를 아예 raw 파일에서 추적하여 삭제
# raw추적해서 삭제하는 이유: 매번 업데이트 때마다 converter업뎃 하는데, 이때 자꾸 재등장하면 안되니깐

def hunting(who, file_name, eluent):
    if who == 'sw':
        path = "/home/ufslab223/Shared/KSW/GPT/data_extracted_gpt"
    elif who =='ain':
        path = '/home/ufslab223/Shared/ains/GPT/data_extracted_gpt'
    file = pd.read_csv(path+'/'+ file_name)
    mask = ((file['eluent1']!=eluent) & (file['eluent2']!=eluent))
    file_ = file[mask]
    file_.reset_index(drop=True, inplace=True)
    file_.to_csv(path+'/'+ file_name, index=False)

hunting('ain', 'datatable522.csv', 'AT')'''

In [5]:
##### eluent를 smiles로 변경하기 전에, elu 이름 표준화 및, smiles, 정보들과 매칭 시스템 #####


#필요 없는 칼럼 지우고 eluent 이름 통일한 후에 통일한 이름 바탕으로, smiles 로 변환하기
elu_list1 = list(LLM_df['elu1'].drop_duplicates())
elu_list2 = list(LLM_df['elu2'].drop_duplicates())
elu_list = list(pd.Series(elu_list1 + elu_list2).drop_duplicates()) # set 대신 이렇게 써야 요소 순서가 고정됨(등장 순서)
elu_list

# 처음 보는 eluent (elu_to_key_df에 없는) 추가되면 대응되는 key 지정해주기
elu_to_key_df = pd.read_csv('elu_to_key_df.csv') 
elu_df = pd.read_csv('eluent_standard.csv')
new_eluent=[]
for elu in elu_list:
    if elu in list(elu_to_key_df['elu_raw']):
        continue
    else:
        new_eluent.append(elu)

# 하나씩 대응되는 표준 key 지정
# eluent_standard_name.csv 에 각 number key와 eluent 표준명을 기재해놓음.
# 만약 여기에 없는 새로운 eluent가 추가되면 number key 하나 추가하여 갱신하면 됨.
elu_key=[]
for elu in new_eluent:
    elu_df = pd.read_csv('eluent_standard.csv') # for 문 내에서 업데이트 된 거 새로 불러오기
    key = int(input(f'{elu} 의 key 지정 --> '))
    elu_key.append(key)
    # 만약 추가되는 eluent가 이름만 새로운게 아니라 새로운 화학종이면 elu_standard에서 추가해놔야함
    if key > len(elu_df):
        std_name = input(f'{key}와 매칭될 표준 이름 입력: ')
        print(key, std_name)
        smiles = input(f'{key}와 매칭될 smiles 입력 (자동 canonicalize 해줄것임): ')
        if smiles == '??':
            cano_smiles = smiles # 논문 작성자가 무슨 용매를 의미한건지 명확하지 않을때 임시로 채워넣는것
        else:
            cano_smiles = Chem.MolToSmiles((Chem.MolFromSmiles(smiles)), canonical = True)
        density = input(f'{key}와 매칭될 밀도 입력: ')

        extra_eleunt_inform = pd.DataFrame({'key': [key],'std_name': [std_name],'567_Smiles': [cano_smiles],'hex_Smiles': [cano_smiles], 'density': [density]})
        elu_df = pd.concat([elu_df, extra_eleunt_inform], axis=0)
        elu_df.to_csv('eluent_standard.csv', index=False) # elu_standard 업데이트 저장


extra_elu2key_df = pd.DataFrame({'elu_raw': new_eluent, 'elu_key': elu_key})
elu_to_key_df = pd.concat([elu_to_key_df, extra_elu2key_df], axis=0)
elu_to_key_df.to_csv('elu_to_key_df.csv', index=False) # elu to key 업데이트 저장

In [ ]:
import sys
sys.path.append('../../../')
from functions import delete_weird_mol_and_None_except_Nan, delete_metal_mol_except_Nan


# ---------------PE_hex 버전

print('1')
LLM_df = delete_weird_mol_and_None_except_Nan(LLM_df, 'Smiles')
print('2')
LLM_df = delete_metal_mol_except_Nan(LLM_df, 'Smiles')
print('3')
LLM_df = delete_weird_mol_and_None_except_Nan(LLM_df, 'elu1')
print('4')
LLM_df = delete_metal_mol_except_Nan(LLM_df, 'elu1')
print('5')
LLM_df = delete_weird_mol_and_None_except_Nan(LLM_df, 'elu2')
print('6')
LLM_df = delete_metal_mol_except_Nan(LLM_df, 'elu2')
print('7')


# ---------------PE_567 버전

print('1')
LLM_df_567 = delete_weird_mol_and_None_except_Nan(LLM_df_567, 'Smiles')
print('2')
LLM_df_567 = delete_metal_mol_except_Nan(LLM_df_567, 'Smiles')
print('3')
LLM_df_567 = delete_weird_mol_and_None_except_Nan(LLM_df_567, 'elu1')
print('4')
LLM_df_567 = delete_metal_mol_except_Nan(LLM_df_567, 'elu1')
print('5')
LLM_df_567 = delete_weird_mol_and_None_except_Nan(LLM_df_567, 'elu2')
print('6')
LLM_df_567 = delete_metal_mol_except_Nan(LLM_df_567, 'elu2')
print('7')

In [6]:
# eluent --> key --> smiles 변환

elu_to_key_df = pd.read_csv('elu_to_key_df.csv') 
elu_to_key_converter = dict(zip(elu_to_key_df['elu_raw'], elu_to_key_df['elu_key']))

elu_df = pd.read_csv('eluent_standard.csv')
key_to_567smiles_converter = dict(zip(elu_df['key'], elu_df['567_Smiles']))
key_to_smiles_converter = dict(zip(elu_df['key'], elu_df['hex_Smiles']))

elu1_list_org = LLM_df['elu1']
elu2_list_org = LLM_df['elu2']

# -----------PE --> hex 버전 ---------------

LLM_df['elu1'] = [key_to_smiles_converter[elu_to_key_converter[name]] for name in elu1_list_org]
LLM_df['elu2']  = [key_to_smiles_converter[elu_to_key_converter[name]] for name in elu2_list_org]

# ratio 칼럼 정리

LLM_df['ratio'] = [rat.replace('/',':',1) if '/' in rat else rat for rat in LLM_df['ratio']]

ratio1 = [float(rat.split(':')[0]) / (float(rat.split(':')[0]) + float(rat.split(':')[1]))
          if isinstance(rat, str) and  ':' in rat else float(1)  for rat in LLM_df['ratio']]
ratio2 = [float(rat.split(':')[1]) / (float(rat.split(':')[0]) + float(rat.split(':')[1]))
          if isinstance(rat, str) and ':' in rat else float(0)  for rat in LLM_df['ratio']]
LLM_df['ratio1'] = ratio1
LLM_df['ratio2'] = ratio2
LLM_df.drop(columns = ['ratio'], inplace=True)
LLM_df = LLM_df[['Smiles', 'elu1', 'elu2', 'ratio1', 'ratio2', 'Rf', 'title', 'doi']]

LLM_df['ratio1'] = LLM_df['ratio1'].apply(lambda x: float(x) if not isinstance(x, float) else x)
LLM_df['ratio2'] = LLM_df['ratio2'].apply(lambda x: float(x) if not isinstance(x, float) else x)
LLM_df['Rf'] = LLM_df['Rf'].apply(lambda x: float(x) if not isinstance(x, float) else x)

#----------PE --> 567버전 ------------------
from copy import deepcopy
LLM_df_567 = deepcopy(LLM_df)
LLM_df_567['elu1'] = [key_to_567smiles_converter[elu_to_key_converter[name]] for name in elu1_list_org]
LLM_df_567['elu2']  = [key_to_567smiles_converter[elu_to_key_converter[name]] for name in elu2_list_org]

In [7]:
LLM_df

,Smiles,elu1,elu2,ratio1,ratio2,Rf,title,doi
0,IC1=CC=C(C=C1)/C=C/C(=O)C1=C(C=CC=C1)C#CC1=CC=...,CCOC(C)=O,CCCCCC,0.005000,0.995000,0.2,Phosphine-Based Porous Organic Polymer-Stabili...,https://pubs.acs.org/doi/10.1021/acs.orglett.5...
1,C1(=CC=CC=C1)C=1C(C2=CC=CC=C2C1C1=CC=CC=C1)=O,CCOC(C)=O,CCCCCC,0.050000,0.950000,0.5,Phosphine-Based Porous Organic Polymer-Stabili...,https://pubs.acs.org/doi/10.1021/acs.orglett.5...
2,COC1=CC=C(C=C1)C=1C(C2=CC=CC=C2C1C1=CC=C(C=C1)...,CCOC(C)=O,CCCCCC,0.050000,0.950000,0.4,Phosphine-Based Porous Organic Polymer-Stabili...,https://pubs.acs.org/doi/10.1021/acs.orglett.5...
3,FC1=CC=C(C=C1)C=1C(C2=CC=CC=C2C1C1=CC=C(C=C1)F)=O,CCOC(C)=O,CCCCCC,0.050000,0.950000,0.4,Phosphine-Based Porous Organic Polymer-Stabili...,https://pubs.acs.org/doi/10.1021/acs.orglett.5...
4,FC(C1=CC=C(C=C1)C=1C(C2=CC=CC=C2C1C1=CC=C(C=C1...,CCOC(C)=O,CCCCCC,0.050000,0.950000,0.4,Phosphine-Based Porous Organic Polymer-Stabili...,https://pubs.acs.org/doi/10.1021/acs.orglett.5...
...,...,...,...,...,...,...,...,...
23971,BrC1=CC=C2C(C(C(=CC2=C1)C(=O)OC(C)C)=O)=[N+]=[N-],CCCCCC,CCOC(C)=O,0.750000,0.250000,0.2,Atroposelective Synthesis of Axially Chiral C2...,https://pubs.acs.org/doi/10.1021/acs.orglett.2...
23972,OC=1C(=CC2=CC=CC=C2C1C=1N(C2=CC(=CC(=C2C1CCC)C...,CCCCCC,CCOC(C)=O,0.952381,0.047619,0.4,Atroposelective Synthesis of Axially Chiral C2...,https://pubs.acs.org/doi/10.1021/acs.orglett.2...
23973,[N+](=[N-])=C1C=2C=CC=C(C2C=CC1=O)C(=O)OC,CCCCCC,CCOC(C)=O,0.833333,0.166667,0.3,Atroposelective Synthesis of Axially Chiral C2...,https://pubs.acs.org/doi/10.1021/acs.orglett.2...
23974,OC=1C(=C2C=CC=C(C2=CC1)C(=O)OC)C=1N(C2=CC(=CC(...,CCCCCC,CCOC(C)=O,0.952381,0.047619,0.2,Atroposelective Synthesis of Axially Chiral C2...,https://pubs.acs.org/doi/10.1021/acs.orglett.2...


In [8]:
LLM_df_567

,Smiles,elu1,elu2,ratio1,ratio2,Rf,title,doi
0,IC1=CC=C(C=C1)/C=C/C(=O)C1=C(C=CC=C1)C#CC1=CC=...,CCOC(C)=O,CCCCCC,0.005000,0.995000,0.2,Phosphine-Based Porous Organic Polymer-Stabili...,https://pubs.acs.org/doi/10.1021/acs.orglett.5...
1,C1(=CC=CC=C1)C=1C(C2=CC=CC=C2C1C1=CC=CC=C1)=O,CCOC(C)=O,CCCCCC,0.050000,0.950000,0.5,Phosphine-Based Porous Organic Polymer-Stabili...,https://pubs.acs.org/doi/10.1021/acs.orglett.5...
2,COC1=CC=C(C=C1)C=1C(C2=CC=CC=C2C1C1=CC=C(C=C1)...,CCOC(C)=O,CCCCCC,0.050000,0.950000,0.4,Phosphine-Based Porous Organic Polymer-Stabili...,https://pubs.acs.org/doi/10.1021/acs.orglett.5...
3,FC1=CC=C(C=C1)C=1C(C2=CC=CC=C2C1C1=CC=C(C=C1)F)=O,CCOC(C)=O,CCCCCC,0.050000,0.950000,0.4,Phosphine-Based Porous Organic Polymer-Stabili...,https://pubs.acs.org/doi/10.1021/acs.orglett.5...
4,FC(C1=CC=C(C=C1)C=1C(C2=CC=CC=C2C1C1=CC=C(C=C1...,CCOC(C)=O,CCCCCC,0.050000,0.950000,0.4,Phosphine-Based Porous Organic Polymer-Stabili...,https://pubs.acs.org/doi/10.1021/acs.orglett.5...
...,...,...,...,...,...,...,...,...
23971,BrC1=CC=C2C(C(C(=CC2=C1)C(=O)OC(C)C)=O)=[N+]=[N-],CCCCC.CCCCCC.CCCCCCC,CCOC(C)=O,0.750000,0.250000,0.2,Atroposelective Synthesis of Axially Chiral C2...,https://pubs.acs.org/doi/10.1021/acs.orglett.2...
23972,OC=1C(=CC2=CC=CC=C2C1C=1N(C2=CC(=CC(=C2C1CCC)C...,CCCCC.CCCCCC.CCCCCCC,CCOC(C)=O,0.952381,0.047619,0.4,Atroposelective Synthesis of Axially Chiral C2...,https://pubs.acs.org/doi/10.1021/acs.orglett.2...
23973,[N+](=[N-])=C1C=2C=CC=C(C2C=CC1=O)C(=O)OC,CCCCC.CCCCCC.CCCCCCC,CCOC(C)=O,0.833333,0.166667,0.3,Atroposelective Synthesis of Axially Chiral C2...,https://pubs.acs.org/doi/10.1021/acs.orglett.2...
23974,OC=1C(=C2C=CC=C(C2=CC1)C(=O)OC)C=1N(C2=CC(=CC(...,CCCCC.CCCCCC.CCCCCCC,CCOC(C)=O,0.952381,0.047619,0.2,Atroposelective Synthesis of Axially Chiral C2...,https://pubs.acs.org/doi/10.1021/acs.orglett.2...


In [9]:
import sys
sys.path.append('../../../')
from functions import delete_weird_mol_and_None_except_Nan, delete_metal_mol_except_Nan


# ---------------PE_hex 버전

print('1')
LLM_df = delete_weird_mol_and_None_except_Nan(LLM_df, 'Smiles')
print('2')
LLM_df = delete_metal_mol_except_Nan(LLM_df, 'Smiles')
print('3')
LLM_df = delete_weird_mol_and_None_except_Nan(LLM_df, 'elu1')
print('4')
LLM_df = delete_metal_mol_except_Nan(LLM_df, 'elu1')
print('5')
LLM_df = delete_weird_mol_and_None_except_Nan(LLM_df, 'elu2')
print('6')
LLM_df = delete_metal_mol_except_Nan(LLM_df, 'elu2')
print('7')


# ---------------PE_567 버전

print('1')
LLM_df_567 = delete_weird_mol_and_None_except_Nan(LLM_df_567, 'Smiles')
print('2')
LLM_df_567 = delete_metal_mol_except_Nan(LLM_df_567, 'Smiles')
print('3')
LLM_df_567 = delete_weird_mol_and_None_except_Nan(LLM_df_567, 'elu1')
print('4')
LLM_df_567 = delete_metal_mol_except_Nan(LLM_df_567, 'elu1')
print('5')
LLM_df_567 = delete_weird_mol_and_None_except_Nan(LLM_df_567, 'elu2')
print('6')
LLM_df_567 = delete_metal_mol_except_Nan(LLM_df_567, 'elu2')
print('7')

1


[16:05:28] Explicit valence for atom # 1 C, 8, is greater than permitted


2742번째 분자 mol=None (smiles가 아닌 값)


[16:05:29] Explicit valence for atom # 1 C, 8, is greater than permitted


4826번째 분자 mol=None (smiles가 아닌 값)


...

4826번째 분자 mol=None (smiles가 아닌 값)


[16:05:48] Explicit valence for atom # 1 C, 8, is greater than permitted


18011번째 분자 mol=None (smiles가 아닌 값)


[16:05:49] Explicit valence for atom # 1 C, 8, is greater than permitted


21437번째 분자 mol=None (smiles가 아닌 값)
2
3830번째 분자 금속원소 포함됨
3831번째 분자 금속원소 포함됨
3832번째 분자 금속원소 포함됨
3833번째 분자 금속원소 포함됨
3834번째 분자 금속원소 포함됨
3835번째 분자 금속원소 포함됨
3836번째 분자 금속원소 포함됨
3837번째 분자 금속원소 포함됨
3838번째 분자 금속원소 포함됨
3839번째 분자 금속원소 포함됨
3840번째 분자 금속원소 포함됨
3841번째 분자 금속원소 포함됨
3842번째 분자 금속원소 포함됨
3843번째 분자 금속원소 포함됨
3844번째 분자 금속원소 포함됨
3845번째 분자 금속원소 포함됨
3846번째 분자 금속원소 포함됨
3847번째 분자 금속원소 포함됨
3848번째 분자 금속원소 포함됨
3849번째 분자 금속원소 포함됨
3850번째 분자 금속원소 포함됨
3851번째 분자 금속원소 포함됨
3852번째 분자 금속원소 포함됨
3853번째 분자 금속원소 포함됨
3854번째 분자 금속원소 포함됨
3855번째 분자 금속원소 포함됨
3856번째 분자 금속원소 포함됨
3857번째 분자 금속원소 포함됨
3858번째 분자 금속원소 포함됨
3859번째 분자 금속원소 포함됨
3860번째 분자 금속원소 포함됨
3861번째 분자 금속원소 포함됨
3862번째 분자 금속원소 포함됨
3863번째 분자 금속원소 포함됨
3864번째 분자 금속원소 포함됨
13272번째 분자 금속원소 포함됨
15814번째 분자 금속원소 포함됨
15815번째 분자 금속원소 포함됨
19109번째 분자 금속원소 포함됨
19110번째 분자 금속원소 포함됨
19111번째 분자 금속원소 포함됨
19112번째 분자 금속원소 포함됨
19113번째 분자 금속원소 포함됨
19114번째 분자 금속원소 포함됨
19115번째 분자 금속원소 포함됨
19116번째 분자 금속원소 포함됨
19117번째 분자 금속원소 포함됨
19118번째 분자 금속원소 포함됨
19119번째 분자 금속원소 포함됨
19120번째 분자 금속원소 포함

In [10]:
#Smiles canonicalize

#-------------PE hex 버전 -------------

LLM_df = LLM_df[(0<LLM_df['Rf']) & (LLM_df['Rf']<=1)]

# ratio1 > ratio2 되도록 정렬 --> 중복 데이터 판별시 필요
switch_ratio_mask = LLM_df['ratio1']-LLM_df['ratio2']<0

LLM_df.loc[switch_ratio_mask, ['ratio1','ratio2']] = LLM_df.loc[switch_ratio_mask, ['ratio2', 'ratio1']].values
LLM_df.loc[switch_ratio_mask, ['elu1','elu2']] = LLM_df.loc[switch_ratio_mask, ['elu2', 'elu1']].values

# compound smiles canonicalize 
LLM_df.loc[:,'Smiles'] = [Chem.MolToSmiles(Chem.MolFromSmiles(smiles), isomericSmiles=True, canonical=True) for smiles in LLM_df['Smiles']]

LLM_df = LLM_df.reset_index(drop=True)


#-------------PE 567 버전 -------------

LLM_df_567 = LLM_df_567[(0<LLM_df_567['Rf']) & (LLM_df_567['Rf']<=1)]

# ratio1 > ratio2 되도록 정렬 --> 중복 데이터 판별시 필요
switch_ratio_mask = LLM_df_567['ratio1']-LLM_df_567['ratio2']<0

LLM_df_567.loc[switch_ratio_mask, ['ratio1','ratio2']] = LLM_df_567.loc[switch_ratio_mask, ['ratio2', 'ratio1']].values
LLM_df_567.loc[switch_ratio_mask, ['elu1','elu2']] = LLM_df_567.loc[switch_ratio_mask, ['elu2', 'elu1']].values

# compound smiles canonicalize 
LLM_df_567.loc[:,'Smiles'] = [Chem.MolToSmiles(Chem.MolFromSmiles(smiles), isomericSmiles=True, canonical=True) for smiles in LLM_df_567['Smiles']]

LLM_df_567 = LLM_df_567.reset_index(drop=True)


### elu1,elu2 도 canonlicalize 완료!

In [11]:
LLM_df['elu1'] = [Chem.MolToSmiles(Chem.MolFromSmiles(s), canonical=True) if pd.notna(s) else np.nan for s in LLM_df['elu1']]
LLM_df['elu2'] = [Chem.MolToSmiles(Chem.MolFromSmiles(s), canonical=True) if pd.notna(s) else np.nan for s in LLM_df['elu2']]

In [12]:
LLM_df_567['elu1'] = [Chem.MolToSmiles(Chem.MolFromSmiles(s), canonical=True) if pd.notna(s) else np.nan for s in LLM_df_567['elu1']]
LLM_df_567['elu2'] = [Chem.MolToSmiles(Chem.MolFromSmiles(s), canonical=True) if pd.notna(s) else np.nan for s in LLM_df_567['elu2']]

### Rf_type 추가:
#### 해당 논문에 정밀한 단위 Rf 있으면 정밀한 논문으로 취급
#### 0.05로 나누었을때 나머지 0 보다 크면 인정해줌

In [13]:
def set_Rf_type(df, not_005_ratio=0.5):
    from decimal import Decimal

    C_Rf_paper_list=[]
    for paper, paper_data in df.groupby(['title'], as_index=True):
        is_it_precise = np.array([Decimal(str(rf)) for rf in paper_data['Rf']]) % Decimal('0.05')
        # if (is_it_precise > 0).any():
        if len(is_it_precise[is_it_precise>0]) > len(is_it_precise)*not_005_ratio:
            C_Rf_paper_list.append(paper[0])

    df['Rf_type'] = ['C' if (title in C_Rf_paper_list) else 'Q' for title in df['title']]
    return df

In [14]:
LLM_df_final = set_Rf_type(LLM_df, 0.7)
LLM_df_567_final = set_Rf_type(LLM_df_567, 0.7)

In [15]:
len(LLM_df_final[LLM_df_final['Rf_type']=='C'])

7591

In [16]:
LLM_df_final.to_csv('../collecting/primary_processed_PE_hex/gpt_PE_hex.csv', index=False)
LLM_df_567_final.to_csv('../collecting/primary_processed_PE_567/gpt_PE_567.csv', index=False)